<a href="https://colab.research.google.com/github/Samuel-Msqt/Stats4Cosmology/blob/main/tutorial1/Cobaya-tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# General python imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
!pip install cobaya

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.9/461.9 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.9/833.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.1 MB/s eta 0:00:00
  Created wheel for cobaya: filename=cobaya-3.5.7-py3-none-any.whl size=524717 sha256=c3c6e6f40ccca766fb5e5631b2b25bc9926db141dd52c7e16bb6420ad01be388
  Stored in directory: /root/.cache/pip/wheels/69/a2/4e/7c7b30ae4a875356d00900a324af003fd5128bbe14d25a6a87
Successfully built cobaya


In [4]:
!cobaya-install cosmo -p .

[install] Installing basic cosmological packages.
[install] Installing external packages at '/content'
[install] The installation path has been written into the global config file: /root/.config/cobaya/config.yaml

theory:camb

[install] Checking if dependencies have already been installed...
[install] Check found no existing installation
[install] (If you expected this to be already installed, re-run `cobaya-install` with --debug to get more verbose output.)
[install] Installing...
[CAMB] Installing pre-requisites...
[CAMB] Downloading camb...
13.3MiB [00:00, 18.6MiB/s]
[CAMB] Downloaded filename CAMB-master.tar.gz
[CAMB] CAMB master downloaded and decompressed correctly.
[CAMB] Compiling camb...
[CAMB] configuration error: `project.license` must be valid exactly by one definition (2 matches found):

    - keys:
        'file': {type: string}
      required: ['file']
    - keys:
        'text': {type: string}
      required: ['text']

DESCRIPTION:
    `Project license <https://peps.py

In [5]:
!pip install camb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.1 MB/s eta 0:00:00


# Attention:

Keep always the Cobaya documentation open: https://cobaya.readthedocs.io

# Exercise 1:

### Use the Metropolis-Hastings MCMC sampler of Cobaya to sample a simple posterior distribution

In this exercise, we aim to sample the posterior distribution corresponding to the product of a gaussian mixture likelihood in Cobaya (see https://cobaya.readthedocs.io/en/latest/likelihood_gaussian_mixture.html) using simple priors for the parameters.

For that, you need to fill up the blocks `likelihood`, `params` and `sampler` of the configuration dictionary and then run Cobaya.

In [1]:
# Fill the configuration dictionary called 'info' below

info = {
# We import the likelihood from Cobaya likelihood libraries
#'likelihood': Cobaya's protected key of the input dictionary:
    "likelihood": {
        "gaussian_mixture": {
            "means": [0.2, 0],
            "covs": [[0.1, 0.05],
                     [0.05, 0.2]],
            "derived": True}},
#'params': Cobaya's protected key of the input dictionary.
# Includes the parameters that the user would like to sample over:
    "params": dict([
        ("a", {
            "prior": {"min": -0.5, "max": 3},
            "latex": r"\alpha"}),
        ("b", {
            "prior": {"dist": "norm", "loc": 0, "scale": 1},
            "ref": 0,
            "proposal": 0.5,
            "latex": r"\beta"}),
        ("hyperparam_a", {
            "latex": r"\alpha^\prime"}),
        ("derived_b", {
            "latex": r"\beta^\prime"})]),
# 'sampler': Cobaya's protected key of the input dictionary.
# you can choose the sampler you want to use.
# Check Cobaya's documentation to see the list of available samplers
    "sampler": {
        "mcmc": None},
# 'force': Cobaya's protected key of the input dictionary.
# 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    "force": True,
# 'debug': Cobaya's protected key of the input dictionary.
# how much information you want Cobaya to print? If debug: True, it prints every single detail
# that is going on internally in Cobaya
    "debug": False,
# 'output': Cobaya's protected key of the input dictionary.
# Where are the results going to be stored, in case that the sampler produce output files?
# For example: chains...
# modify the path below within 'output' to choose a name and a directory for those files
    "output": "chains_exercise1"}

In [ ]:
#  We import Cobaya
from cobaya.run import run
# Now we run!
updated_info, sampler = run(info)

### Plot the corresponding distribution using GetDist

Do not forget to open the GetDist html DEMO: https://getdist.readthedocs.io/en/latest/plot_gallery.html

In [ ]:
# Import corresponding modules from GetDist
from getdist.mcsamples import loadMCSamples, MCSamplesFromCobaya
import getdist.plots as gdplt

# Load the chains and the updated config file from the previous run
gd_sample = MCSamplesFromCobaya(updated_info, sampler.products()["sample"])

# Analyze the chains (print the mean and covmat according to GetDist documentation)
mean = ...
covmat = ...
print("Mean:")
print(mean)
print("Covariance matrix:")
print(covmat)

# PLOT THE CHAINS (have a look at the DEMO above from GetDist)
# The goal is to produce a triangle plot

### Now, let's change the likelihood above by a new gaussian ring likelihood defined on cartesian coordinates

Cobaya allows to pass self-defined likelihoods as likelihoods of the `likelihood` block.

In [ ]:
# We define the new likelihood as a python function
# ATTENTION: Cobaya always needs to return the loglike!

def gauss_ring_logp(x, y, mean_radius=1, std=0.02):
    """
    Defines a gaussian ring likelihood on cartesian coordinater,
    around some ``mean_radius`` and with some ``std``.
    """
    return stats.norm.logpdf(np.sqrt(x**2 + y**2), loc=mean_radius, scale=std)

In [ ]:
# Let's update the new likelihood in the info dictionary above

info = {"likelihood": {"ring": ...}}

In [ ]:
# The new likelihood requests two new parameters: x and y
# Therefore, we need to define them in the params block
# of the info dictionary above adding a new prior

# Use uniform distributions as priors for x and y between 0 and 2

info["params"] = {...}

In [ ]:
# We add the polar coordinates theta and radius as derived parameters within the params block

def get_r(x, y):
    return np.sqrt(x ** 2 + y ** 2)


def get_theta(x, y):
    return np.arctan(y / x)

# Update the params part of the info dictionary with these derived params

In [ ]:
# We add some new conditions in the sampler block, to give extra tries in case the mcmc gets stuck

info["sampler"] = {"mcmc": {"Rminus1_stop": 0.001, "max_tries": 1000}}

# Re-run cobaya as we did above!

### Replot the chains using GetDist

We aim to create two triangle plots: one for the cartesian coordinates x and y, and another for the polar coordinates theta and radius

In [ ]:
# Load the chains and the updated config file from the previous run
# Make two triangle plots: one for the cartesian coordinates x and y,
# and a different triangle plot for the radius and angle theta

# Exercise 2:

### Make an evaluation of the full posterior of the parameters of $\Lambda$CDM using BAO + SN + PLANCK18

In this exercise, we aim to make an evaluation of the posterior distribution of the Standard Cosmological Model parameters using BAO + SN + PLANCK18 data.

For that, you need to create a new config file speciying the corresponding likelihoods, the parameters of the $\Lambda$CDM model and specify the sampler to `evaluate`.

What about the `theory` module? Do you need to specify it? What would you choose?

If you feel like working, you can change the sampler `evaluate` to the sampler `mcmc` to explore the full posterior distribution. Attention: it will take around 3 or 4 days. Let it run for one night so that tomorrow you can have a look at the chains.

- Check this link to see how you can define the new corresponding config file: https://cobaya.readthedocs.io/en/latest/cosmo_basic_runs.html
- All the corresponding likelihood names are within the section “COSMOLOGICAL LIKELIHOODS” at https://cobaya.readthedocs.io/en/latest/

In [ ]:
# You need to start by defining a new config python dictionary
# (or if you feel brave, you can write a script like a yaml format
# and dump it as a python dictionary)

# LCDM has 6 parameters, so you need to define at least 6 parameters
# to be sampled from within the `params` block

info = {...}